In [1]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA, make_loss
from functions import *
from scipy.linalg import hadamard

In [2]:
n_samples = 1000
n_test = 100
n = 10
p = 8
rng = np.random.RandomState(0)

fit_loss = 'l2'
reg = 'l1'

fit_function, der_function = {
                              'l2': (l2_fit, l2_der),
                              'logreg': (logreg_fit, logreg_der)
                              }[fit_loss]
reg_function, prox = {
                      'l2': (l2_pen, l2_prox),
                      'l1': (l1_pen, l1_prox),
                      None: (no_pen, no_prox)
                      }[reg]
loss = make_loss(fit_function, reg_function)


H = hadamard(p) / np.sqrt(p)
#S = p / np.arange(1, p + 1)
#S **= 2
S = np.ones(p)
K = np.dot(H, S[:, None] * H.T)

z_true = rng.randn(p, n_samples)
D = rng.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = rng.randn(n, n_samples)

z_test = rng.randn(p, n_test)
eps_test = rng.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if fit_loss == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1

In [3]:
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = 0.01 * lbda_max
print(X.shape)

(10, 1000)


In [6]:
n_layers = 3
lista = LISTA(D, lbda, n_layers, fit_loss, reg, variables='both', learn_levels=True)
print(lista.compute_loss(X))
lista.fit(X, l_rate=1e-1, max_iter=1000)
print(lista.compute_loss(X))

2.8669445539805727
1.8098525668141432
